In [6]:
import pandas as pd
import numpy as np
import gspread
import talib as ta
pd.options.mode.chained_assignment = None  # default='warn'

In [7]:
import datetime as dt
df_nyse_holidays = pd.read_csv("nyse_holidays.csv")
nyse_holidays = [dt.datetime.strptime(date, "%d-%b-%y").date() for date in df_nyse_holidays['Date']]

In [8]:
from datetime import date
from tiingo import TiingoClient

In [9]:
config = {}

# To reuse the same HTTP Session across API calls (and have better performance), include a session key.
config['session'] = True

# If you don't have your API key as an environment variable,
# pass it in via a configuration dictionary.
config['api_key'] = '4faa12a9eeb4da65c763258897ab593545f98f00'
#cp "5d58df57a94d667b0fdbb57d81828680e1d8f14a"
#gmail main "6e0dfa8bf5a18b3a5167ad5fdc8f2f438c136b18"

# Initialize
client = TiingoClient(config)

In [10]:
# function to get previous trading day
from datetime import date, timedelta
def prev_weekday(adate):
    adate -= timedelta(days=1)
    while ((adate.weekday() > 4) & (adate not in nyse_holidays)):
        # Mon-Fri are 0-4 #also add holidays to this
        adate -= timedelta(days=1)
    return adate

# str(prev_weekday(date.today()))

In [11]:
def trend_(dataframe,days = 20):
    dataframe['Trend+'] = (dataframe['Smooth_+DX']>dataframe['Smooth_-DX']).astype(int)
    dataframe['Trend-'] = (dataframe['Smooth_-DX']>dataframe['Smooth_+DX']).astype(int)
    if(np.sum(dataframe['Trend+'])==len(dataframe['Trend+'])):
        return('uptrend')
    elif(np.sum(dataframe['Trend-'])==len(dataframe['Trend-'])):
        return('downtrend')
    else:
        return('')

In [12]:
from itertools import compress
from candle_rankings import candle_rankings
def recognize_candlestick(df):
    candles = ta.get_function_groups()['Pattern Recognition']
    exclude_items = ('CDLCOUNTERATTACK',
                     'CDLLONGLINE',
                     'CDLSHORTLINE',
                     'CDLSTALLEDPATTERN',
                     'CDLKICKINGBYLENGTH')
    candle_names = [candle for candle in candles if candle not in exclude_items]
    op = df['Open']
    hi = df['High']
    lo = df['Low']
    cl = df['Close']

    for candle in candle_names:
        df[candle] = getattr(ta, candle)(op, hi, lo, cl)


    df['candlestick_pattern'] = np.nan
    df['candlestick_match_count'] = np.nan
    for index, row in df.iterrows():
        # no pattern found
        if len(row[candle_names]) - sum(row[candle_names] == 0) == 0:
            df.loc[index,'candlestick_pattern'] = ""
            df.loc[index, 'candlestick_match_count'] = 0
        # single pattern found
        elif len(row[candle_names]) - sum(row[candle_names] == 0) == 1:
            # bull pattern 100 or 200
            if any(row[candle_names].values > 0):
                pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
                df.loc[index, 'candlestick_pattern'] = pattern
                df.loc[index, 'candlestick_match_count'] = 1
            # bear pattern -100 or -200
            else:
                pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
                df.loc[index, 'candlestick_pattern'] = pattern
                df.loc[index, 'candlestick_match_count'] = 1
        # multiple patterns matched -- select best performance
        else:
            # filter out pattern names from bool list of values
            patterns = list(compress(row[candle_names].keys(), row[candle_names].values != 0))
            container = []
            for pattern in patterns:
                if row[pattern] > 0:
                    container.append(pattern + '_Bull')
                else:
                    container.append(pattern + '_Bear')
            rank_list = [candle_rankings[p] for p in container]
            if len(rank_list) == len(container):
                rank_index_best = rank_list.index(min(rank_list))
                df.loc[index, 'candlestick_pattern'] = container[rank_index_best]
                df.loc[index, 'candlestick_match_count'] = len(container)
    # clean up candle columns
    cols_to_drop = candle_names
    df.drop(cols_to_drop, axis = 1, inplace = True)

    return df

In [13]:
def get_rsi_infer(rsi):
    if(rsi>70):
        return "Overbought"
    elif((rsi>50) & (rsi<=70)):
        return "Bullish"
    elif((rsi<=50) & (rsi>30)):
        return "Bearish"
    elif(rsi<=30):
        return "Oversold"

In [14]:
def get_stock_data(stock):
    stock = stock.upper()
    a = client.get_dataframe(stock, startDate=str(prev_weekday(date.today())-timedelta(weeks=52)),
                                      endDate=str(date.today()), 
                                      frequency='daily')
    a['Stock Name'] =stock
    
    a = a[['Stock Name','adjClose', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume']]
    a.columns = ['Stock Name','Close', 'High', 'Low', 'Open', 'Volume']
    a.columns = ['Stock Name','Close', 'High', 'Low', 'Open', 'Volume']
    inputs_ta = {
        'open': a['Open'],
        'high': a['Close'],
        'low': a['Low'],
        'close': a['Close'],
        'volume': a['Volume']
    }
    a['RSI'] = ta.RSI(inputs_ta['close'], timeperiod=14)
    a['RSI_infer'] = a['RSI'].apply(lambda x: get_rsi_infer(x))
    a['upper_bollinger'], a['middle_bollinger'], a['lower_bollinger'] = ta.BBANDS(inputs_ta['close'], 
                                                                                     matype=ta.MA_Type.T3)
    a['MACD'],a['macdsignal'], a['macdhist'] = ta.MACD(inputs_ta['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    a.drop(['macdsignal', 'macdhist'], axis=1,inplace= True)
    a['Plus_DI']=ta.PLUS_DI(inputs_ta['high'], inputs_ta['low'], inputs_ta['close'], timeperiod=14)
    a['Minus_DI']=ta.MINUS_DI(inputs_ta['high'], inputs_ta['low'], inputs_ta['close'], timeperiod=14)
    a['ADX'] = ta.ADX(inputs_ta['high'], inputs_ta['low'], inputs_ta['close'], timeperiod=14)
    a['CCI'] = ta.CCI(inputs_ta['high'], inputs_ta['low'], inputs_ta['close'], timeperiod=14)
    a['WILLR'] = ta.WILLR(inputs_ta['high'], inputs_ta['low'], inputs_ta['close'], timeperiod=14)
    a['Prev Close'] = a['Close'].shift(1)
    a['high_minus_low'] = a['High'] - a['Low']
    a['high_minus_prev_close'] = a['High'] - a['Prev Close']
    a['low_minus_prev_close'] = a['Low'] - a['Prev Close']
    a['Prev High'] = a['High'].shift(1)
    a['High_minus_Prev_High'] = a['High'] - a['Prev High']
    a['Prev Low'] = a['Low'].shift(1)
    a['Prev_Low_minus_Low'] = a['Prev Low'] - a['Low']

    a['+DX'] = a.apply(lambda x: x['High_minus_Prev_High'] if((x['High_minus_Prev_High']>x['Prev_Low_minus_Low']) & 
                                                              (x['High_minus_Prev_High']>0)) else 0, axis = 1)
    a['-DX'] = a.apply(lambda x: x['Prev_Low_minus_Low'] if((x['Prev_Low_minus_Low']>x['High_minus_Prev_High']) & 
                                                            (x['Prev_Low_minus_Low']>0)) else 0, axis = 1)
    a['Smooth_+DX'] = a['+DX'].rolling(window = 14, min_periods = 14).mean()
    a['Smooth_-DX'] = a['-DX'].rolling(window = 14, min_periods = 14).mean()
    trend_stock = trend_(a.tail(20))
    
    b = a.iloc[-1]
    temp = pd.DataFrame(b.T.reset_index().T.values[1:],columns = b.T.reset_index().T.values[0])
    b['Candlestick_Pattern'] = recognize_candlestick(temp[['Open','High','Low','Close']])['candlestick_pattern'].values[0]

    b['Trend'] = trend_stock
    b['20_D_Avg_Vol'] = np.average(a.tail(20)['Volume'])
    b['10_D_Avg_Vol'] = np.average(a.tail(10)['Volume'])
    b['5_D_Avg_Vol'] = np.average(a.tail(5)['Volume'])

    b['200_D_Avg_Close'] = np.average(a.tail(200)['Close'])
    b['50_D_Avg_Close'] = np.average(a.tail(50)['Close'])
    b['20_D_Avg_Close'] = np.average(a.tail(20)['Close'])
    b['5_D_Avg_Close'] = np.average(a.tail(5)['Close'])
    
    b['Prv_200_D_Avg_Close'] = np.average(a.tail(201)[1:200]['Close'])
    b['Prv_50_D_Avg_Close'] = np.average(a.tail(51)[1:50]['Close'])
    b['Prv_20_D_Avg_Close'] = np.average(a.tail(21)[1:20]['Close'])

    
    b['120_D_Max_Close'] = np.max(a.tail(120)['Close'])
    b['20_D_Max_Close'] = np.max(a.tail(20)['Close'])
    b['5_D_Max_Close'] = np.average(a.tail(5)['Close'])

    b['Yr_H_Price'] = np.max(a['High'])
    b['Yr_L_Price'] = np.min(a['Low'])
    b['Yr_H_Vol'] = np.max(a['Volume'])
    b['Yr_L_Vol'] = np.min(a['Volume'])
    
    b['Prv_Yr_H_Price'] = np.max(a[:-1]['High'])
    b['Prv_Yr_L_Price'] = np.min(a[:-1]['Low'])
    b['Prv_Yr_H_Vol'] = np.max(a[:-1]['Volume'])
    b['Prv_Yr_L_Vol'] = np.min(a[:-1]['Volume'])
    
    b['Prv_2_Close'] = a.iloc[-3]['Close']
    b['Prv_3_Close'] = a.iloc[-4]['Close']
    
    b['Prv_1_Open'] = a.iloc[-2]['Open']
    b['Prv_2_Open'] = a.iloc[-3]['Open']
    b['Prv_3_Open'] = a.iloc[-4]['Open']

    b['Prv_1_Low'] = a.iloc[-2]['Low']
    b['Prv_2_Low'] = a.iloc[-3]['Low']
    b['Prv_3_Low'] = a.iloc[-4]['Low']
    b['Prv_4_Low'] = a.iloc[-5]['Low']
    b['Prv_5_Low'] = a.iloc[-6]['Low']
    b['Prv_6_Low'] = a.iloc[-7]['Low']
    b['Prv_7_Low'] = a.iloc[-8]['Low']
    b['Prv_8_Low'] = a.iloc[-9]['Low']
    b['10_D_Min_Low'] = np.min(a.tail(10)['Low'])
    b['50_D_Min_Low'] = np.min(a.tail(50)['Low'])

    b['Prv_1_High'] = a.iloc[-2]['High']
    b['Prv_2_High'] = a.iloc[-3]['High']
    b['Prv_3_High'] = a.iloc[-4]['High']
    b['Prv_4_High'] = a.iloc[-5]['High']
    b['Prv_5_High'] = a.iloc[-6]['High']
    b['Prv_6_High'] = a.iloc[-7]['High']
    b['Prv_7_High'] = a.iloc[-8]['High']
    b['Prv_8_High'] = a.iloc[-9]['High']
    b['10_D_Max_High'] = np.max(a.tail(10)['High'])
    b['50_D_Max_High'] = np.max(a.tail(50)['High'])
    
    b['100_D_Avg_Range'] = np.mean(a.tail(100)['High']-a.tail(100)['Low'])
    b['Prev_Volume'] = a.iloc[-2]['Volume']
    
    b = pd.DataFrame(b.T.reset_index().T.values[1:],columns = b.T.reset_index().T.values[0])
    return b

In [15]:
# df_ = pd.DataFrame()
# df_['Open'] = [307.71]
# df_['High'] = [307.9]
# df_['Low'] = [300.21]
# df_['Close'] = [300.35]

## US Stocks

In [16]:
us_stocks = pd.read_excel("us_stocks.xlsx",header=0,sheet_name="us_stocks")

In [33]:
import requests
token = 'AAHwf08vrNDxf_zEl-Ri0hDESHvGM_WIaok'
method = 'sendMessage'
response = requests.post(
    url='https://api.telegram.org/bot1216810091:{0}/{1}'.format(token, method),
    data={'chat_id': -444135572, 'text': "US stock scanning started for {0}".format(str(date.today()))}).json()
# :AAHwf08vrNDxf_zEl-Ri0hDESHvGM_WIaok/sendMessage 

In [18]:
from tqdm.notebook import tqdm

stock_data_us = pd.DataFrame(columns = ['Stock Name', 'Open','High','Low','Close','Prev Close',
                                         'Yr_H_Price','Yr_L_Price','Trend',
                                         '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                                         'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                                         '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                                         '50_D_Max_High','10_D_Max_High',
                                         'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                                         '50_D_Min_Low','10_D_Min_Low',
                                         'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                                         '100_D_Avg_Range',
                                         'Yr_H_Vol','Yr_L_Vol',
                                         '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                                         'Volume','Prev_Volume',
                                         'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                                         'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                                         'Prv_8_High','Prv_8_Low',
                                        'Candlestick_Pattern', 'RSI',
                                        'upper_bollinger','middle_bollinger','lower_bollinger',
                                        'Plus_DI','Minus_DI','ADX','MACD','CCI','WILLR'
                                       ])

stock_names = us_stocks['Symbol']

for stock in tqdm(stock_names):
    try:
        temp = get_stock_data(stock)
        temp = temp[['Stock Name', 'Open','High','Low','Close','Prev Close',
                     'Yr_H_Price','Yr_L_Price','Trend',
                     '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                     'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                     '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                     '50_D_Max_High','10_D_Max_High',
                     'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                     '50_D_Min_Low','10_D_Min_Low',
                     'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                     '100_D_Avg_Range',
                     'Yr_H_Vol','Yr_L_Vol',
                     '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                     'Volume','Prev_Volume',
                     'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                     'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                     'Prv_8_High','Prv_8_Low',
                     'Candlestick_Pattern', 'RSI','upper_bollinger','middle_bollinger','lower_bollinger',
                     'Plus_DI','Minus_DI','ADX','MACD','CCI','WILLR'
                    ]]
        stock_data_us=stock_data_us.append(temp, ignore_index = True)
    except Exception as e:
        response = requests.post(
            url='https://api.telegram.org/bot1216810091:{0}/{1}'.format(token, method),
            data={'chat_id': -444135572, 'text': str(e)}).json()
#         print(e)
#         print(stock)

HBox(children=(FloatProgress(value=0.0, max=451.0), HTML(value='')))

In [19]:
# get_stock_data('AXBKY')

In [20]:
us_stocks_data = pd.merge(stock_data_us, us_stocks, how='left', left_on=['Stock Name'], 
                                   right_on = [us_stocks.columns[0]])

In [21]:
# us_stocks_data

In [22]:
# # to convert dataframe to list for uploading to google sheet
us_stocks_data = us_stocks_data[['Stock Name', 'Stock', 'Open','High','Low','Close','Prev Close',
                                 'Yr_H_Price','Yr_L_Price','Trend',
                                 '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                                 'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                                 '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                                 '50_D_Max_High','10_D_Max_High',
                                 'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                                 '50_D_Min_Low','10_D_Min_Low',
                                 'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                                 '100_D_Avg_Range',
                                 'Yr_H_Vol','Yr_L_Vol',
                                 '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                                 'Volume','Prev_Volume',
                                 'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                                 'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                                 'Prv_8_High','Prv_8_Low',
                                 'Candlestick_Pattern', 'RSI','upper_bollinger','middle_bollinger','lower_bollinger',
                                 'Plus_DI','Minus_DI','ADX','MACD','CCI','WILLR']]
us_stocks_data.rename(columns={"Stock Name": "Symbol"},inplace=True)
us_stocks_data.replace(np.nan,'',inplace = True)

us_stocks_data.loc[us_stocks_data['Symbol']=='BRK-A','Symbol'] = 'BRKA'
us_stocks_data.loc[us_stocks_data['Symbol']=='BRK-B','Symbol'] = 'BRK.B'

In [23]:
us_stocks_data.to_excel(excel_writer="us_stocks_data.xlsx")

## Writing to Google Sheet

In [24]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient import discovery
import pandas as pd
import numpy as np
import gspread
pd.options.mode.chained_assignment = None  # default='warn'

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('stock-scanner-project-75f48f4873c1.json',scope)
service = discovery.build('sheets', 'v4', credentials=credentials)
gs = service.spreadsheets()
spreadsheet_id = '1AkRMRjrD09PL06ZtjDvT5aeuNkbGMyM-Z8jQ01XC7F0'

In [25]:
us_stocks_data = us_stocks_data.T.reset_index().T.values.tolist()

In [26]:
## writing list to google sheet
response_data = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption='USER_ENTERED',
        range='US_Stocks',
        body=dict(
            majorDimension='ROWS',
            values=us_stocks_data)
        ).execute()

In [27]:
## writing updated time to google sheet
import datetime
response_datetime = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption='USER_ENTERED',
        range='US_Stocks!BQ1',
        body={
            'values': [[datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]]
        }
).execute()

In [29]:
requests.post(
    url='https://api.telegram.org/bot1216810091:{0}/{1}'.format(token, method),
    data={'chat_id': -444135572, 'text': "US stock scanning succesful for {0}!".format(str(date.today()))}
)

<Response [404]>